# Example for usage ARc Policy via a Strands Agent

Install requirements

In [1]:
%pip install -r requirements.txt --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
# If needed, this will allow for the restart of the notebook

from IPython.display import display_html
def restart_kernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)
restart_kernel()


## Setup for the agent

In [3]:
from strands import Agent
from strands.models import BedrockModel
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
from pydantic import BaseModel
import boto3
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
from findings_utils import extract_reasoning_findings
import logging
from strands_tools import retrieve
import os

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
StrandsTelemetry().setup_console_exporter()

# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used

INFO | strands.telemetry.config | Initializing tracer
INFO | strands.telemetry.config | Enabling console export


## Provide values for the following cell

In [4]:
# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ARN = "arn:aws:bedrock:us-east-1:161615149547:automated-reasoning-policy/malxiyr0ojy2"
GUARDRAIL_ID = "an852wptcjol"
GUARDRAIL_VERSION = "4"
KNOWLEDGE_BASE_ID = "CZDJXI9C4E"
# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0
# MODEL_ID = "us.amazon.nova-lite-v1:0"
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"


In [5]:
# Setup the environment for the agent and tool
# Allow for the metadata to be retrieved on sources from the KB
os.environ['RETRIEVE_ENABLE_METADATA_DEFAULT'] = 'true'
# Allow for the retrieve tool to interact with the KB
os.environ['KNOWLEDGE_BASE_ID'] = KNOWLEDGE_BASE_ID

In [6]:

# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str, arc_policy_arn: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.arc_policy_arn = arc_policy_arn
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.claim_valid = True
        self.findings = ''
        self.policy_definition = {}
        self.before_tool_event_flag = False
        self.before_model_event_flag = False

        if self.arc_policy_arn:
            try:
                bedrock_client = boto3.client('bedrock')
                response = bedrock_client.export_automated_reasoning_policy_version(policyArn=self.arc_policy_arn)
                self.policy_definition = response.get('policyDefinition', {})
            except Exception as e:
                print(f"Error getting policy definition: {str(e)}")
                raise

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeModelCallEvent, self.before_model_event)
        registry.add_callback(BeforeToolCallEvent, self.before_tool_event)
        registry.add_callback(MessageAddedEvent, self.message_added)

    def message_added(self, event: MessageAddedEvent) -> None:
        if self.before_tool_event_flag:
            # Since a tool was called, just ignore this message addition
            self.before_tool_event_flag = False
            return
        
        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        if event.message.get("role") == "user":
            # Store the input for later usage and allow the loop to continue to process
            self.input = content
            return

        if not content:
            return
            #do something 

        # Capture if this is the first time that findings will be created
        first_findings = (not self.findings)

        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]
        print ("HERE LOOKIE HERE",content_to_validate)
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is correct
        self.findings = extract_reasoning_findings(response, self.policy_definition)
         
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            self.claim_valid = False

        # Add information to the output
        if self.findings and first_findings:
            new_output = content
            new_output = new_output + f"\n\nfindings: {self.findings}"
            new_output = new_output + f"\n\nclaim_valid: {self.claim_valid}"
            event.message["content"][0]["text"] = new_output
        
    def before_model_event(self, event: BeforeModelCallEvent) -> None:
        self.before_model_event_flag = True

    def before_tool_event(self, event: BeforeToolCallEvent) -> None:
        self.before_tool_event_flag = True

# Create structured output
class StructuredOutputModel(BaseModel):
    claim_valid: bool
    content: str
    findings: str

# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findings aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

agent_instructions="""You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract.
    
You will be provided with JSON data that has claim information and vehicle damage information, you should:
1. Extract from the JSON data required claim information to be validated
2. Focus on time of event and time of claim creation
3. Focus on claims and coverage inconsistencies

Your responses should :
- If your response is "Valid claim", then output the provide the full JSON structure provided from the input
- If you response is "Invalid claim", then output the response "This Claim is Invalid and no appraisal nor settlement is required"
- If you response is "Invalid claim", then provide clear explanation on why is invalid in the output
- In cases where a clear outcome is not present, recommend the user to check with their insurance agent directly. 

Take your time to think though the answer and evalute carefully."

"""


# Create agent with the guardrail-protected model
auto_policy_agent = Agent(
    system_prompt=agent_instructions,
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION, ARC_POLICY_ARN)],
    tools=[retrieve]
)



DEBUG | strands.models.bedrock | config=<{'model_id': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'include_tool_result_status': 'auto'}> | initializing
DEBUG | strands.models.bedrock | region=<us-east-1> | bedrock client created
DEBUG | strands.tools.loader | tool_name=<retrieve>, module=<retrieve> | loading tools from module
DEBUG | strands.tools.registry | tool_name=<retrieve>, tool_type=<python>, is_dynamic=<False> | registering tool
DEBUG | strands.tools.registry | tools_dir=</home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/tools> | found tools directory
DEBUG | strands.tools.registry | tools_dir=</home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/tools> | scanning
DEBUG | strands.tools.registry | tool_modules=<[]> | discovered
DEBUG | strands.tools.registry | tool_count=<0>, success_count=<0> | finished loading tools


In [7]:
# Ask the agent a question
auto_policy_agent("I was delivering goods for my company using my personal car. I got into an accident, this was over 3 months ago. Does my policy cover this?")

DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<retrieve> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<1> | tools configured
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<retrieve> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<1> | tools configured
INFO | strands.telemetry.metrics | Creating Strands MetricsClient
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<retrieve> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<1> | tools configured
DEBUG | strands.event_loop.streaming | model=<<strands.models.bedrock.BedrockModel object at 0x7f08f7f4f590>> | streaming messages
DEBUG | strands.models.bedrock | formatting request
DEBUG | strands.models.bedrock | request=<{'modelId': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'messages': [{'content': [{'text': 'I was 

I'll help you determine if your policy covers this situation. Let me search for information about commercial use of personal vehicles and coverage details.
Tool #1: retrieve


DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.event_loop._retry | stop_reason=<tool_use> | model call succeeded, resetting retry state


{
    "name": "chat",
    "context": {
        "trace_id": "0x810daf8c367210466dfc764fab81f2ea",
        "span_id": "0x44a8cefb0e67680a",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0xef2d39c13eb3916a",
    "start_time": "2026-02-13T17:42:16.561448Z",
    "end_time": "2026-02-13T17:42:18.231298Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "gen_ai.event.start_time": "2026-02-13T17:42:16.561450+00:00",
        "gen_ai.operation.name": "chat",
        "gen_ai.system": "strands-agents",
        "gen_ai.request.model": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "gen_ai.event.end_time": "2026-02-13T17:42:18.231191+00:00",
        "gen_ai.usage.prompt_tokens": 1511,
        "gen_ai.usage.input_tokens": 1511,
        "gen_ai.usage.completion_tokens": 87,
        "gen_ai.usage.output_tokens": 87,
        "gen_ai.usage.total_tokens": 1598,
        "gen_ai.server.time_to_first_token": 819,
        "gen_ai

DEBUG | strands.tools.executors._executor | tool_use=<{'toolUseId': 'tooluse_jjXZIh3WiKpouI6xo63bDl', 'name': 'retrieve', 'input': {'text': 'personal car commercial use delivery business coverage exclusions'}}> | streaming
DEBUG | strands.tools.registry | getting tool configurations
DEBUG | strands.tools.registry | tool_name=<retrieve> | loaded tool config
DEBUG | strands.tools.registry | tool_count=<1> | tools configured


...

DEBUG | strands.models.bedrock | finished streaming response from model
DEBUG | strands.event_loop._retry | stop_reason=<end_turn> | model call succeeded, resetting retry state


 business use of your personal vehicle.{
    "name": "chat",
    "context": {
        "trace_id": "0x810daf8c367210466dfc764fab81f2ea",
        "span_id": "0xe166c26edce60742",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x411cee7c469e8d63",
    "start_time": "2026-02-13T17:42:40.858281Z",
    "end_time": "2026-02-13T17:42:49.052661Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "gen_ai.event.start_time": "2026-02-13T17:42:40.858283+00:00",
        "gen_ai.operation.name": "chat",
        "gen_ai.system": "strands-agents",
        "gen_ai.request.model": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "gen_ai.event.end_time": "2026-02-13T17:42:49.052553+00:00",
        "gen_ai.usage.prompt_tokens": 18916,
        "gen_ai.usage.input_tokens": 18916,
        "gen_ai.usage.completion_tokens": 449,
        "gen_ai.usage.output_tokens": 449,
        "gen_ai.usage.total_tokens": 19365,
        "gen_ai.serve

DEBUG | strands.agent.conversation_manager.sliding_window_conversation_manager | message_count=<6>, window_size=<40> | skipping context reduction


{
    "name": "invoke_agent Strands Agents",
    "context": {
        "trace_id": "0x810daf8c367210466dfc764fab81f2ea",
        "span_id": "0x32f835ff101fe57f",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2026-02-13T17:42:16.560614Z",
    "end_time": "2026-02-13T17:42:56.711437Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "gen_ai.event.start_time": "2026-02-13T17:42:16.560621+00:00",
        "gen_ai.operation.name": "invoke_agent",
        "gen_ai.system": "strands-agents",
        "gen_ai.agent.name": "Strands Agents",
        "gen_ai.request.model": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "gen_ai.agent.tools": "[\"retrieve\"]",
        "system_prompt": "You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract.\n    \nYou will be provided 

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'Based on my analysis of your policy contract, I can provide you with the following assessment:\n\n## **This Claim is Invalid and no appraisal nor settlement is required**\n\n### **Reasons for Claim Invalidity:**\n\n**1. Commercial Use Exclusion:**\nYour policy contains multiple clear exclusions that apply to your situation. The policy explicitly excludes coverage when your personal vehicle is being used "for retail or wholesale delivery, including, but not limited to, the pickup, transport or delivery of magazines, newspapers, mail or food."\n\nThis exclusion appears in several sections:\n- **Part I (Liability Coverage):** Excludes bodily injury or property damage when using a covered auto "to carry persons or property for compensation or a fee" or "for retail or wholesale delivery"\n- **Part II (Medical Payments):** Same exclusions apply\n- **Part III (Uninsured/Underinsured Motorist):** Same exclu

In [8]:
# Get the structured output.  Without providing a prompt, the agent will act on conversation history to get the response
response = auto_policy_agent.structured_output(StructuredOutputModel)

# Extract the findings from the response
print(f"response.claim_valid = {response.claim_valid}")
print(f"repsonse.content = {response.content}")
print(f"repsonse.findings = {response.findings}")

/tmp/ipykernel_206905/3153824652.py:2: DeprecationWarning: Agent.structured_output method is deprecated. You should pass in `structured_output_model` directly into the agent invocation. see: https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/structured-output/
  response = auto_policy_agent.structured_output(StructuredOutputModel)
DEBUG | strands.models.bedrock | formatting request
DEBUG | strands.models.bedrock | request=<{'modelId': 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'messages': [{'content': [{'text': 'I was delivering goods for my company using my personal car. I got into an accident, this was over 3 months ago. Does my policy cover this?'}], 'role': 'user'}, {'content': [{'text': "I'll help you determine if your policy covers this situation. Let me search for information about commercial use of personal vehicles and coverage details.\n\nfindings: # Automated Reasoning Policy Findings\n\n## Finding 1\n**Finding Type:** Translationambiguous\n\n#


Tool #3: StructuredOutputModel


DEBUG | strands.models.bedrock | finished streaming response from model


{
    "name": "execute_structured_output",
    "context": {
        "trace_id": "0x6134ba1f4196458659400999f91ee817",
        "span_id": "0x297d2935478c0ec4",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": null,
    "start_time": "2026-02-13T17:42:56.718466Z",
    "end_time": "2026-02-13T17:43:02.957180Z",
    "status": {
        "status_code": "UNSET"
    },
    "attributes": {
        "gen_ai.system": "strands-agents",
        "gen_ai.agent.name": "Strands Agents",
        "gen_ai.agent.id": "default",
        "gen_ai.operation.name": "execute_structured_output"
    },
    "events": [
        {
            "name": "gen_ai.system.message",
            "timestamp": "2026-02-13T17:42:56.718540Z",
            "attributes": {
                "role": "system",
                "content": "[{\"text\": \"You are an expert automotive claims validaiton specialist that determines if the users auto insurance claim is valid based on the provided information and 